In [1]:
from htmd import *
htmd.config(viewer='vmd')
# %qtconsole
#os.chdir('/webdata/73hboiwia98hdj209jq0/')  # Skip this command.


Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049

You are on the latest HTMD version (1.5.3).



# System Building Trypsin-Benzamidine

by [Stefan Doerr](https://github.com/stefdoerr)
&nbsp;

&nbsp;

**Download all the required files for the tutorial from this** [link](http://pub.htmd.org/73hboiwia98hdj209jq0/bentryp.tar.gz)

You can watch the presentation here:

[![](http://pub.htmd.org/73hboiwia98hdj209jq0/bentryp_youtube.png)](https://youtu.be/DF9cHKBX19A?t=5m42s)

## Obtain structures

In [2]:
from glob import *
glob('presentations-2016/bentryp/*')

[]

In [3]:
prot = Molecule('3PTB')
#prot = Molecule('bentryp/trypsin.pdb')
#prot.view()

2016-11-09 11:17:21,921 - htmd.molecule.molecule - INFO - Using local copy for 3PTB: /home/ec2-user/miniconda3/lib/python3.5/site-packages/htmd/data/pdb/3ptb.pdb


## Clean structures

Water molecules and one calcium ion present in the PDB crystal structure were also read by HTMD. Start by removing the ligand:

In [4]:
prot.filter('chain A and (protein or water or resname CA)')
prot.view()

2016-11-09 11:17:22,027 - htmd.molecule.molecule - INFO - Removed 9 atoms. 1692 atoms remaining in the molecule.


Alternatively:

In [5]:
prot.remove('resname BEN')

2016-11-09 11:17:24,105 - htmd.molecule.molecule - INFO - Removed 0 atoms. 1692 atoms remaining in the molecule.


array([], dtype=int32)

## Define segments

To build a system in HTMD, we need to separate the chemical molecules into separate segments. This prevents the builder from accidentally bonding different chemical molecules and allows us to add caps to them.

In [6]:
prot.set('segid', 'P', sel='protein')
prot.set('segid', 'W', sel='water')
prot.set('segid', 'CA', sel='resname CA')

Center the protein to the origin

In [7]:
prot.center()

## Let's add a ligand!

In [8]:
ligand = Molecule('bentryp/benzamidine.pdb')
ligand.center()
ligand.view(sel='resname MOL', style='Licorice')

But the ligand is located inside the protein...
We would like the ligand to be:

* At a certain distance from the protein
* Rotated randomly, to provide different starting conditions

## Let's reposition the ligand then

In [9]:
from htmd.molecule.util import uniformRandomRotation
ligand.rotateBy(uniformRandomRotation())

This took care of the ligand rotation. But now we still need to position it far from the protein.
We need to find out the radius of the protein:

![](http://pub.htmd.org/73hboiwia98hdj209jq0/maxdist.png)

In [10]:
from htmd.molecule.util import maxDistance
D = maxDistance(prot, 'all')
print(D)

28.1477526263


In [11]:
D += 10
ligand.moveBy([D, 0, 0])  # Move the ligand 10 Angstrom away from the furthest protein atom in X dimension
ligand.rotateBy(uniformRandomRotation())

## Don't forget to add a segment

In [12]:
ligand.set('segid','L')
ligand.set('resname','MOL')

### Mix it all together

In [13]:
mol = Molecule(name='combo')
mol.append(prot)
mol.append(ligand)
mol.reps.add(sel='protein', style='NewCartoon', color='Secondary Structure')
mol.reps.add(sel='resname MOL', style='Licorice')
mol.view()

## Solvate

> Water is the driving force of all nature.
>
> -Leonardo da Vinci

![](http://pub.htmd.org/73hboiwia98hdj209jq0/waterbox.png)

In [14]:
D = D + 5
smol = solvate(mol, minmax=[[-D, -D, -D], [D, D, D]])
smol.reps.add(sel='water', style='Lines')
smol.view()

2016-11-09 11:17:28,340 - htmd.builder.solvate - INFO - Using water pdb file at: /home/ec2-user/miniconda3/lib/python3.5/site-packages/htmd/builder/wat.pdb
2016-11-09 11:17:29,127 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2


Solvating: 100% (8/8) [############################################] eta 00:01 /


2016-11-09 11:17:42,853 - htmd.builder.solvate - INFO - After removing water molecules colliding with other molecules, 19287 water molecules were added to the system.


## Build

In [15]:
charmm.listFiles()

---- Topologies files list: /home/ec2-user/miniconda3/lib/python3.5/site-packages/htmd/builder/charmmfiles/top/ ----
top/top_all22_prot.rtf
top/top_all22star_prot.rtf
top/top_all35_ethers.rtf
top/top_all36_carb.rtf
top/top_all36_cgenff.rtf
top/top_all36_lipid.rtf
top/top_all36_na.rtf
top/top_all36_prot.rtf
top/top_water_ions.rtf
---- Parameters files list: /home/ec2-user/miniconda3/lib/python3.5/site-packages/htmd/builder/charmmfiles/par/ ----
par/par_all22_prot.prm
par/par_all22star_prot.prm
par/par_all35_ethers.prm
par/par_all36_carb.prm
par/par_all36_cgenff.prm
par/par_all36_lipid.prm
par/par_all36_na.prm
par/par_all36_prot.prm
par/par_all36_prot_mod.prm
par/par_water_ions.prm
---- Stream files list: /home/ec2-user/miniconda3/lib/python3.5/site-packages/htmd/builder/charmmfiles/str/ ----
str/carb/toppar_all36_carb_glycolipid.str
str/carb/toppar_all36_carb_glycopeptide.str
str/carb/toppar_all36_carb_model.str
str/lipid/toppar_all36_lipid_bacterial.str
str/lipid/toppar_all36_lipid_car

## Build and ionize

In [16]:
topos  = ['top/top_all22star_prot.rtf', 'bentryp/benzamidine.rtf']
params = ['par/par_all22star_prot.prm', 'bentryp/benzamidine.prm']

molbuilt = charmm.build(smol, topo=topos, param=params, outdir='03b_out_build')

molbuilt.view(sel='protein', style='NewCartoon', 
              color='Secondary Structure', hold=True)
molbuilt.view(sel='resname MOL', style='Licorice', hold=True)
molbuilt.view(sel='ions', style='VDW', hold=True)
molbuilt.view(sel='water', style='Lines')

2016-11-09 11:17:56,336 - htmd.builder.charmm - INFO - Writing out segments.


Bond between A: [serial 48 resid 22 resname CYS chain A segid P]
             B: [serial 1007 resid 157 resname CYS chain A segid P]

Bond between A: [serial 185 resid 42 resname CYS chain A segid P]
             B: [serial 298 resid 58 resname CYS chain A segid P]

Bond between A: [serial 811 resid 128 resname CYS chain A segid P]
             B: [serial 1521 resid 232 resname CYS chain A segid P]

Bond between A: [serial 853 resid 136 resname CYS chain A segid P]
             B: [serial 1327 resid 201 resname CYS chain A segid P]

Bond between A: [serial 1084 resid 168 resname CYS chain A segid P]
             B: [serial 1190 resid 182 resname CYS chain A segid P]

Bond between A: [serial 1265 resid 191 resname CYS chain A segid P]
             B: [serial 1422 resid 220 resname CYS chain A segid P]



2016-11-09 11:18:39,532 - htmd.builder.builder - INFO - 6 disulfide bonds were added
2016-11-09 11:18:39,638 - htmd.builder.charmm - INFO - Starting the build.
2016-11-09 11:18:40,161 - htmd.builder.charmm - WARNING - Failed to set coordinates for 1 atoms.
2016-11-09 11:18:40,162 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 137 atoms.
2016-11-09 11:18:40,163 - htmd.builder.charmm - WARNING - Please check /home/ec2-user/03_PreparationBuilding_TGiorgino/03b_build/log.txt for further information.
2016-11-09 11:18:40,163 - htmd.builder.charmm - INFO - Finished building.
2016-11-09 11:18:43,370 - htmd.builder.ionize - INFO - Adding 9 anions + 0 cations for neutralizing and 0 ions for the given salt concentration.
2016-11-09 11:18:43,828 - htmd.builder.ionize - INFO - Min distance of ions from molecule: 5A
2016-11-09 11:18:43,829 - htmd.builder.ionize - INFO - Min distance between ions: 5A
2016-11-09 11:18:43,829 - htmd.builder.ionize - INFO - Placing 9 ions.
2016-11-09 1

In [17]:
molbuilt

Molecule with 61277 atoms and 1 frames
Atom field - altloc shape: (61277,)
Atom field - atomtype shape: (61277,)
Atom field - beta shape: (61277,)
Atom field - chain shape: (61277,)
Atom field - charge shape: (61277,)
Atom field - coords shape: (61277, 3, 1)
Atom field - element shape: (61277,)
Atom field - insertion shape: (61277,)
Atom field - masses shape: (61277,)
Atom field - name shape: (61277,)
Atom field - occupancy shape: (61277,)
Atom field - record shape: (61277,)
Atom field - resid shape: (61277,)
Atom field - resname shape: (61277,)
Atom field - segid shape: (61277,)
Atom field - serial shape: (61277,)
angles shape: (25258, 3)
bonds shape: (41964, 2)
box shape: (3, 1)
boxangles shape: (3, 1)
dihedrals shape: (8659, 4)
fileloc shape: (1, 2)
impropers shape: (583, 4)
reps: 
ssbonds shape: (0,)
step shape: (0,)
time shape: (0,)
topoloc: /home/ec2-user/03_PreparationBuilding_TGiorgino/03b_build/structure.psf
viewname: structure